# Comparação de gêneros no OBRAS entre PALAVRAS e UDPIPE

## Abrindo corpoPALAVRAS e corpoOBRAS

In [1]:
import re
with open("corpoPALAVRAS", encoding="latin-1") as f:
    PALAVRAS = f.read()
with open("corpoOBRAS", encoding="latin-1") as f:
    OBRAS = f.read()

### Resultado da leitura dos arquivos (7 primeiras linhas)

In [2]:
PALAVRAS.splitlines()[0:7]

['<obra id="A_Mortalha_de_Alzira Prosa:romance AA 1894 naturalismo_realismo_romantismo masc ">',
 '<tituloobra id="A Mortalha de Alzira">',
 '<autor id="Aluísio Azevedo">',
 '<u>',
 '<s>',
 '<t>',
 'A\to\tDET_artd\t0\tS\tF\t>N\t0\t0\t1->2']

In [3]:
OBRAS.splitlines()[0:7]

['<obra id="A_Mortalha_de_Alzira Prosa:romance AA 1894 naturalismo_realismo_romantismo masc ">',
 '<tituloobra id="A Mortalha de Alzira">',
 '<autor id="Aluísio Azevedo">',
 '<u>',
 '<s>',
 '<t>',
 'A\tA_Mortalha_de_Alzira\tProsa:romance\tAluAze\t1894\tnaturalismo_realismo_romantismo\tmasc\torig\to\tDET_artd\t0\tS\tF\t>N\t0\t0\t0\t1->2\tmORTALHA\t__UNDEF__\t__UNDEF__\t__UNDEF__']

## Criando estrutura de dados

In [21]:
PALAVRAS_obras = [{
        'obra': re.findall('<obra id="([^"]*)"', x),
        'tituloobra': re.findall('<tituloobra id="([^"]*)"', x),
        'autor': re.findall('<autor id="([^"]*)"', x),
        'sentences': [[{
                    'forma': z.split("\t")[0],
                    'lema': z.split("\t")[1],
                    'pos': z.split("\t")[2],
                    'genero': z.split("\t")[5]
                } for z in y.split("\n") if len(z.split("\t")) > 5
                      ] for y in re.findall('<u>(.*?)</u>', x, flags=re.DOTALL) if '\t' in y]
        } for x in re.findall('<obra.*?</obra>', PALAVRAS, flags=re.DOTALL)]

In [19]:
OBRAS_obras = [{
        'obra': re.findall('<obra id="([^"]*)"', x),
        'tituloobra': re.findall('<tituloobra id="([^"]*)"', x),
        'autor': re.findall('<autor id="([^"]*)"', x),
        'sentences': [[{
                    'forma': z.split("\t")[0],
                    'lema': z.split("\t")[8],
                    'pos': z.split("\t")[9],
                    'genero': z.split("\t")[12]
                } for z in y.split("\n") if len(z.split("\t")) > 5
                      ] for y in re.findall('<u>(.*?)</u>', x, flags=re.DOTALL) if '\t' in y]
        } for x in re.findall('<obra.*?</obra>', OBRAS, flags=re.DOTALL)]

### Estrutura pronta da primeira das obras

In [7]:
print(PALAVRAS_obras[0]['autor'])
print(PALAVRAS_obras[0]['obra'])
print(PALAVRAS_obras[0]['tituloobra'])
PALAVRAS_obras[0]['sentences'][0]

['Aluísio Azevedo']
['A_Mortalha_de_Alzira Prosa:romance AA 1894 naturalismo_realismo_romantismo masc ']
['A Mortalha de Alzira']


[{'forma': 'A', 'genero': 'F', 'lema': 'o', 'pos': 'DET_artd'},
 {'forma': 'MORTALHA', 'genero': 'F', 'lema': 'mORTALHA', 'pos': 'N'},
 {'forma': 'DE', 'genero': '0', 'lema': 'de', 'pos': 'PRP'},
 {'forma': 'ALZIRA', 'genero': 'F', 'lema': 'Alzira', 'pos': 'PROP'}]

In [8]:
print(OBRAS_obras[0]['autor'])
print(OBRAS_obras[0]['obra'])
print(OBRAS_obras[0]['tituloobra'])
OBRAS_obras[0]['sentences'][0]

['Aluísio Azevedo']
['A_Mortalha_de_Alzira Prosa:romance AA 1894 naturalismo_realismo_romantismo masc ']
['A Mortalha de Alzira']


[{'forma': 'A', 'genero': 'F', 'lema': 'o', 'pos': 'DET_artd'},
 {'forma': 'MORTALHA', 'genero': 'F', 'lema': 'mORTALHA', 'pos': 'N'},
 {'forma': 'DE', 'genero': '0', 'lema': 'de', 'pos': 'PRP'},
 {'forma': 'ALZIRA', 'genero': 'F', 'lema': 'Alzira', 'pos': 'PROP'}]

## Comparando PALAVRAS e OBRAS

### Número de obras no PALAVRAS e no OBRAS

In [22]:
print(len(PALAVRAS_obras))
print(len(OBRAS_obras))

311
311


#### Para corrigir a diferença, buscar obras que existem em um e não no outro

In [23]:
print([x for x in [y['tituloobra'] for y in PALAVRAS_obras] if x not in [z['tituloobra'] for z in OBRAS_obras]])
print([x for x in [y['tituloobra'] for y in OBRAS_obras] if x not in [z['tituloobra'] for z in PALAVRAS_obras]])

[['Capital federal']]
[['Sonetos e poemas']]


#### Eliminar a obra ['Capital federal'] do corpoOBRAS e verificar resultado

In [24]:
PALAVRAS_obras = [x for x in PALAVRAS_obras if x['tituloobra'][0] != 'Capital federal']
OBRAS_obras = [x for x in OBRAS_obras if x['tituloobra'][0] != 'Sonetos e poemas']
print(len(PALAVRAS_obras))
print(len(OBRAS_obras))

310
310


### Buscar obras fora de ordem entre PALAVRAS e OBRAS

In [25]:
fora_de_ordem = False
for i in range(len(PALAVRAS_obras)):
    if PALAVRAS_obras[i]['tituloobra'] != OBRAS_obras[i]['tituloobra']:
        fora_de_ordem = True
        print(f"{PALAVRAS_obras[i]['tituloobra']} / {OBRAS_obras[i]['tituloobra']}")
        
if not fora_de_ordem:
    print("Não há obras fora de ordem")

Não há obras fora de ordem


### Buscar obras com número de sentenças diferente

In [26]:
for i in range(len(PALAVRAS_obras)):
    if len(PALAVRAS_obras[i]['sentences']) != len(OBRAS_obras[i]['sentences']):
        print(f"{i} - {PALAVRAS_obras[i]['tituloobra']}")
        print(f"PALAVRAS ({len(PALAVRAS_obras[i]['sentences'])}) / ACDC ({len(OBRAS_obras[i]['sentences'])})")

12 - ['A dança dos ossos']
PALAVRAS (310) / ACDC (318)
51 - ['A Alma Encantadora das Ruas']
PALAVRAS (4030) / ACDC (4029)
57 - ['A Moreninha']
PALAVRAS (3070) / ACDC (3069)
93 - ['A semana']
PALAVRAS (16251) / ACDC (16250)


#### Apagar obras com número de sentenças diferente

In [27]:
OBRAS_obras = [x for n, x in enumerate(OBRAS_obras) if n not in [12,51,57,93]]
PALAVRAS_obras = [x for n,x in enumerate(PALAVRAS_obras) if n not in [12,51,57,93]]
print(len(OBRAS_obras))

for i in range(len(PALAVRAS_obras)):
    if len(PALAVRAS_obras[i]['sentences']) != len(OBRAS_obras[i]['sentences']):
        print(f"{i} - {PALAVRAS_obras[i]['tituloobra']}")
        print(f"PALAVRAS ({len(PALAVRAS_obras[i]['sentences'])}) / ACDC ({len(OBRAS_obras[i]['sentences'])})")

306


### Buscar sentenças com números de tokens diferentes
além dos erros de atomização, decidir o que fazer com as demais diferenças

In [31]:
output = ""
diff_list = {'OBRAS': {}, 'PALAVRAS': {}}
n_sentences = 0
for i in range(len(PALAVRAS_obras)):
    for k in range(len(OBRAS_obras[i]['sentences'])):
        text_PALAVRAS = " ".join([x['forma'] for x in PALAVRAS_obras[i]['sentences'][k]])
        text_OBRAS = " ".join([x['forma'] for x in OBRAS_obras[i]['sentences'][k]])
        if len(text_PALAVRAS.split()) != len(text_OBRAS.split()):
            output += text_PALAVRAS
            output += "\n" + text_OBRAS
            output += "\n\n"
            lista_PALAVRAS = [x['forma'] for x in PALAVRAS_obras[i]['sentences'][k]]
            lista_OBRAS = [x['forma'] for x in OBRAS_obras[i]['sentences'][k]]
            for diff in lista_PALAVRAS:
                if diff not in lista_OBRAS:
                    diff_list['OBRAS'][diff] = diff_list['OBRAS'][diff] + 1 if diff in diff_list['OBRAS'] else 1
            for diff in lista_OBRAS:
                if diff not in lista_PALAVRAS:
                    diff_list['PALAVRAS'][diff] = diff_list['PALAVRAS'][diff] + 1 if diff in diff_list['PALAVRAS'] else 1

import pprint
#for corpus in diff_list:
    #print(f"\n{len(diff_list[corpus])} palavras que foram suprimidas de {corpus}\n")
    #pprint.pprint(sorted(diff_list[corpus].items(), key=lambda x: x[1], reverse=True))
print(f"\n{len([x for x in output.splitlines() if x])/2} sentenças com tamanho diferente\n")
print(output)
with open("lista_suprimidas.txt", "w") as f:
    f.write(output)


1040.0 sentenças com tamanho diferente

-- Conclue- se destes trocadilhos , que sairei daqui sem ouvir uma palavra de esperança ...
-- Conclue-se destes trocadilhos , que sairei daqui sem ouvir uma palavra de esperança ...

João Romão não saia nunca a passeio , nem ia à missa aos domingos ; tudo que rendia a sua venda e mais a quitanda seguia direitinho para a caixa econômica e daí então para o banco .
João Romão não nunca a passeio , nem ia à missa aos domingos ; tudo que rendia a sua venda e mais a quitanda seguia direitinho para a caixa econômica e daí então para o banco .

Quando saia à rua punha um eterno vestido de seda preta , , cuja saia não fazia rugas , e um xale encarnado que lhe dava a todo o corpo um feitio piramidal .
Quando saia à rua punha um eterno vestido de seda preta , achamalotada , cuja saia não fazia rugas , e um xale encarnado que lhe dava a todo o corpo um feitio piramidal .

E depois de um jogo complicado de reis , valetes e damas , que ela dispunha sobre a m

### Sentenças com quaisquer tokens (formas) diferentes
(inútil, né?)

In [51]:
output = ""
n_sentences = 0
for i in range(len(PALAVRAS_obras)):
    for k in range(len(OBRAS_obras[i]['sentences'])):
        text_PALAVRAS = " ".join([x['forma'] for x in PALAVRAS_obras[i]['sentences'][k]])
        text_OBRAS = " ".join([x['forma'] for x in OBRAS_obras[i]['sentences'][k]])
        if text_PALAVRAS != text_OBRAS:
            output += text_PALAVRAS
            output += "\n" + text_OBRAS
            output += "\n\n"
            
print(f"Resultado: {len([x for x in output.splitlines() if x])/2} sentenças com qualquer forma diferente\n")
print(output)

Resultado: 1373.0 sentenças com qualquer forma diferente

Qual seria a mãe , tão néscia , que , procurando passar seu filho às mãos de quem o pudesse fazer viver , fosse procurar um lugar onde eram crime a voz e o choro desses anjinhos da terra ? ...
Qual seria a mãe , tão néscia , que , procurando passar seu filho às mãos de quem o pudesse fazer viver , fosse procurar um lugar onde crime a voz e o choro desses anjinhos da terra ? ...

ângelo , coitado , nada conhecia disso nem por notícia sequer ; como igualmente não conhecia o outro gênero de pregadores , não menos comum nesse tempo , o do pregador terrível , de pulso forte e cabeça dura , que ia para o púlpito de cacete escondido debaixo do capote , e cujos sermões eram por via de regra uma descarga política e uma tremenda descompostura , contra o partido dos Jansenistas ou contra o partido dos Molinistas , conforme a filiação do orador , e que , em geral , acabavam também por soluços e gemidos , mas estes agora bem sinceros e bem r